In [1]:
from keras.datasets import mnist
import numpy as np
np.random.seed(10)

from time import time
import numpy as np
import keras.backend as K
from keras.engine.topology import Layer, InputSpec

from keras.models import Model
from keras.optimizers import SGD
from keras import callbacks
from keras.initializers import VarianceScaling
from sklearn.cluster import KMeans
from sklearn import metrics
from keras import layers
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape, Conv2DTranspose

from encoder_function import autoencoder, autoencoderConv2D_1, autoencoderConv2D_2, ClusteringLayer


Using TensorFlow backend.


## Load Data

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))
x = x.reshape((x.shape[0], -1))
x = np.divide(x, 255.)
n_clusters = len(np.unique(y))
x.shape

(70000, 784)

## Baseline K Means Clustering Accuracy

In [3]:
kmeans = KMeans(n_clusters=n_clusters, n_init=20, n_jobs=-1)
y_pred_kmeans = kmeans.fit_predict(x)

In [4]:
metrics.adjusted_rand_score(y,y_pred_kmeans)

0.3651913455607028

## Pre-train Auto-encoder

In [5]:
dims = [x.shape[-1], 500, 500, 2000, 10]
init = VarianceScaling(scale=1. / 3., mode='fan_in',distribution='uniform')
pretrain_optimizer = SGD(lr=1, momentum=0.9)
pretrain_epochs = 300
batch_size = 256
save_dir = './results'


Instructions for updating:
Colocations handled automatically by placer.


In [6]:
autoencoder1, encoder1 = autoencoder(dims, init=init)
autoencoder1.compile(optimizer=pretrain_optimizer, loss='mse')
# autoencoder.fit(x, x, batch_size=batch_size, epochs=pretrain_epochs) #, callbacks=cb)
# autoencoder.save_weights(save_dir + '/ae_weights.h5')
autoencoder1.load_weights(save_dir + '/ae_weights.h5')

## Clustering Layer

In [7]:
clustering_layer = ClusteringLayer(n_clusters, name='clustering')(encoder1.output)
model = Model(inputs=encoder1.input, outputs=clustering_layer)
model.compile(optimizer=SGD(0.01, 0.9), loss='kld')


### step1. initialize cluster centers with kmeans

In [8]:
kmeans = KMeans(n_clusters=n_clusters, n_init=20)
y_pred = kmeans.fit_predict(encoder1.predict(x))

y_pred_last = np.copy(y_pred)
print('Clustering ARI after encoding:%f'% metrics.adjusted_rand_score(y, y_pred_last))

Clustering ARI after encoding:0.825019


In [9]:
model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])

### step2.Deep Clustering

In [10]:
# computing an auxiliary target distribution
def target_distribution(q):
    weight = q ** 2 / q.sum(0)
    return (weight.T / weight.sum(1)).T

loss = 0
index = 0
maxiter = 8000
update_interval = 140
index_array = np.arange(x.shape[0])
tol = 0.001 # tolerance threshold to stop training

# start training

# for ite in range(int(maxiter)):
#     if ite % update_interval == 0:
#         q = model.predict(x, verbose=0)
#         p = target_distribution(q)  # update the auxiliary target distribution p

#         # evaluate the clustering performance
#         y_pred = q.argmax(1)
#         if y is not None:
#             acc = np.round(metrics.acc(y, y_pred), 5)
#             nmi = np.round(metrics.nmi(y, y_pred), 5)
#             ari = np.round(metrics.ari(y, y_pred), 5)
#             loss = np.round(loss, 5)
#             print('Iter %d: acc = %.5f, nmi = %.5f, ari = %.5f' % (ite, acc, nmi, ari), ' ; loss=', loss)

#         # check stop criterion - model convergence
#         delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
#         y_pred_last = np.copy(y_pred)
#         if ite > 0 and delta_label < tol:
#             print('delta_label ', delta_label, '< tol ', tol)
#             print('Reached tolerance threshold. Stopping training.')
#             break
#     idx = index_array[index * batch_size: min((index+1) * batch_size, x.shape[0])]
#     loss = model.train_on_batch(x=x[idx], y=p[idx])
#     index = index + 1 if (index + 1) * batch_size <= x.shape[0] else 0

# model.save_weights(save_dir + '/DEC_model_final.h5')

In [11]:
model.load_weights(save_dir + '/DEC_model_final.h5')
# Eval.
q = model.predict(x, verbose=0)
p = target_distribution(q)  # update the auxiliary target distribution p

# evaluate the clustering performance
y_pred = q.argmax(1)
if y is not None:
    nmi = np.round(metrics.adjusted_mutual_info_score(y, y_pred), 5)
    ari = np.round(metrics.adjusted_rand_score(y, y_pred), 5)
    loss = np.round(loss, 5)
    print('nmi = %.5f, ari = %.5f' % (nmi, ari), ' ; loss=', loss)


('nmi = 0.90869, ari = 0.92286', ' ; loss=', 0)


/usr/local/lib/python2.7/dist-packages/sklearn/metrics/cluster/supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


## Experiment 2. Convolutional Auto-encoder

In [12]:
def autoencoderConv2D_1(input_shape=(28, 28, 1), filters=[32, 64, 128, 10]):
    input_img = Input(shape=input_shape)
    if input_shape[0] % 8 == 0:
        pad3 = 'same'
    else:
        pad3 = 'valid'
    x = Conv2D(filters[0], 5, strides=2, padding='same', activation='relu', name='conv1', input_shape=input_shape)(input_img)

    x = Conv2D(filters[1], 5, strides=2, padding='same', activation='relu', name='conv2')(x)

    x = Conv2D(filters[2], 3, strides=2, padding=pad3, activation='relu', name='conv3')(x)

    x = Flatten()(x)
    encoded = Dense(units=filters[3], name='embedding')(x)
    x = Dense(units=filters[2]*int(input_shape[0]/8)*int(input_shape[0]/8), activation='relu')(encoded)

    x = Reshape((int(input_shape[0]/8), int(input_shape[0]/8), filters[2]))(x)
    x = Conv2DTranspose(filters[1], 3, strides=2, padding=pad3, activation='relu', name='deconv3')(x)

    x = Conv2DTranspose(filters[0], 5, strides=2, padding='same', activation='relu', name='deconv2')(x)

    decoded = Conv2DTranspose(input_shape[2], 5, strides=2, padding='same', name='deconv1')(x)
    return Model(inputs=input_img, outputs=decoded, name='AE'), Model(inputs=input_img, outputs=encoded, name='encoder')

autoencoder2, encoder2 = autoencoderConv2D_1(input_shape=(28, 28, 1), filters=[32, 64, 128, 10])
pretrain_epochs = 100
batch_size = 256
autoencoder2.compile(optimizer='adadelta', loss='mse')
# autoencoder2.fit(x, x, batch_size=batch_size, epochs=pretrain_epochs)
# autoencoder2.save_weights(save_dir+'/conv_ae_weights.h5')

In [13]:
autoencoder2.load_weights(save_dir+'/conv_ae_weights.h5')
clustering_layer = ClusteringLayer(n_clusters, name='clustering')(encoder2.output)
model = Model(inputs=encoder2.input, outputs=clustering_layer)
model.compile(optimizer='adam', loss='kld')

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))
x = x.reshape(x.shape + (1,))
x = np.divide(x, 255.)

kmeans = KMeans(n_clusters=n_clusters, n_init=20)
y_pred = kmeans.fit_predict(encoder2.predict(x))
y_pred_last = np.copy(y_pred)
print ('Convolutional Auto-encoder 1 has ARI: %f' % metrics.adjusted_rand_score(y,y_pred_last))

Convolutional Auto-encoder 1 has ARI: 0.638219


In [14]:
model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])
loss = 0
index = 0
maxiter = 8000
update_interval = 140
index_array = np.arange(x.shape[0])
tol = 0.001 # tolerance threshold to stop training

# start training
# for ite in range(int(maxiter)):
#     if ite % update_interval == 0:
#         q = model.predict(x, verbose=0)
#         p = target_distribution(q)  # update the auxiliary target distribution p

#         # evaluate the clustering performance
#         y_pred = q.argmax(1)
#         if y is not None:
#             acc = np.round(metrics.acc(y, y_pred), 5)
#             nmi = np.round(metrics.nmi(y, y_pred), 5)
#             ari = np.round(metrics.ari(y, y_pred), 5)
#             loss = np.round(loss, 5)
#             print('Iter %d: acc = %.5f, nmi = %.5f, ari = %.5f' % (ite, acc, nmi, ari), ' ; loss=', loss)

#         # check stop criterion
#         delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
#         y_pred_last = np.copy(y_pred)
#         if ite > 0 and delta_label < tol:
#             print('delta_label ', delta_label, '< tol ', tol)
#             print('Reached tolerance threshold. Stopping training.')
#             break
#     idx = index_array[index * batch_size: min((index+1) * batch_size, x.shape[0])]
#     loss = model.train_on_batch(x=x[idx], y=p[idx])
#     index = index + 1 if (index + 1) * batch_size <= x.shape[0] else 0

# model.save_weights(save_dir + '/conv_DEC_model_final.h5')

#Load the clustering model trained weights
model.load_weights(save_dir + '/conv_DEC_model_final.h5')
# Eval.
q = model.predict(x, verbose=0)
p = target_distribution(q)  # update the auxiliary target distribution p

# evaluate the clustering performance
y_pred = q.argmax(1)
if y is not None:
    nmi = np.round(metrics.adjusted_mutual_info_score(y, y_pred), 5)
    ari = np.round(metrics.adjusted_rand_score(y, y_pred), 5)
    loss = np.round(loss, 5)
    print('nmi = %.5f, ari = %.5f' % (nmi, ari), ' ; loss=', loss)



('nmi = 0.81327, ari = 0.75605', ' ; loss=', 0)


## Experiment 3. Model to train Autoencoder and clustering layer together (Conv)

In [15]:
autoencoder, encoder = autoencoderConv2D_1()
autoencoder.load_weights(save_dir+'/conv_ae_weights.h5')
clustering_layer = ClusteringLayer(n_clusters, name='clustering')(encoder.output)
model = Model(inputs=encoder.input, outputs=[clustering_layer, autoencoder.output])

In [16]:
kmeans = KMeans(n_clusters=n_clusters, n_init=20)
y_pred = kmeans.fit_predict(encoder.predict(x))
model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])
y_pred_last = np.copy(y_pred)

In [17]:
model.compile(loss=['kld', 'mse'], loss_weights=[0.1, 1], optimizer='adam')
model.load_weights(save_dir + '/conv_b_DEC_model_final.h5')


In [18]:
q, _ = model.predict(x, verbose=0)
p = target_distribution(q)  # update the auxiliary target distribution p

# evaluate the clustering performance
y_pred = q.argmax(1)
if y is not None:
    nmi = np.round(metrics.adjusted_mutual_info_score(y, y_pred), 5)
    ari = np.round(metrics.adjusted_rand_score(y, y_pred), 5)
    loss = np.round(loss, 5)
    print('nmi = %.5f, ari = %.5f' % (nmi, ari), ' ; loss=', loss)



('nmi = 0.82206, ari = 0.76617', ' ; loss=', 0)


## Experiment 4. Model to train Autoencoder and clustering layer together (Fully connected)

In [19]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))
x = x.reshape((x.shape[0], -1))
x = np.divide(x, 255.)
n_clusters = len(np.unique(y))
x.shape

(70000, 784)

In [20]:
dims = [x.shape[-1], 500, 500, 2000, 10]
init = VarianceScaling(scale=1. / 3., mode='fan_in',
                           distribution='uniform')
pretrain_optimizer = SGD(lr=1, momentum=0.9)
pretrain_epochs = 300
batch_size = 256
save_dir = './results'

from encoder_function import autoencoder
autoencoder, encoder = autoencoder(dims, init=init)
autoencoder.load_weights(save_dir+'/ae_weights.h5')
clustering_layer = ClusteringLayer(n_clusters, name='clustering')(encoder.output)
model = Model(inputs=encoder.input,
            outputs=[clustering_layer, autoencoder.output])

In [21]:
kmeans = KMeans(n_clusters=n_clusters, n_init=20)
y_pred = kmeans.fit_predict(encoder.predict(x))
model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])
y_pred_last = np.copy(y_pred)

In [22]:
model.compile(loss=['kld', 'mse'], loss_weights=[0.1, 1], optimizer=pretrain_optimizer)

model.load_weights(save_dir + '/b_DEC_model_final.h5')
q, _ = model.predict(x, verbose=0)
p = target_distribution(q)  # update the auxiliary target distribution p

# evaluate the clustering performance
y_pred = q.argmax(1)
if y is not None:
    nmi = np.round(metrics.adjusted_mutual_info_score(y, y_pred), 5)
    ari = np.round(metrics.adjusted_rand_score(y, y_pred), 5)
    loss = np.round(loss, 5)
    print('nmi = %.5f, ari = %.5f' % (nmi, ari), ' ; loss=', loss)



('nmi = 0.90365, ari = 0.91366', ' ; loss=', 0)
